In [11]:
import queue
from experiment_information import *
from experiments.sweep_generalization_error import *
from _version import __version__
from data import *
import pandas as np
import numpy as np
import logging
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Create a SweepExperiment
experiment = get_default_experiment()
# use json dump to save the experiment parameters
with open("sweep_experiment.json","w") as f:
    # use the NumpyEncoder to encode numpy arrays
    json.dump(experiment.__dict__,f,cls=NumpyEncoder)

In [46]:
df_experiments = None
df_state_evolution = None
df_erm = None
logger = logging.getLogger()
with DatabaseHandler(logger,"experiments/experiments.db") as dbHandler:
    df_experiments = dbHandler.get_experiments()
    df_state_evolution = dbHandler.get_state_evolutions()
    df_erm = dbHandler.get_erms()


In [47]:
# get the completed experiments with the current code version ordered by date
df_experiments = df_experiments[(df_experiments["completed"]==True) & (df_experiments["code_version"]==__version__)]
df_experiments = df_experiments.sort_values(by="date",ascending=False)
df_experiments.head(2)

,experiment_id,experiment_name,duration,code_version,date,state_evolution_repetitions,erm_repetitions,alphas,epsilons,lambdas,tau,d,erm_methods,completed
9,e771baf8-dd50-467e-9262-2346ca9f0a0d,Default Experiment,43.604981,7,2023-03-25 12:43:11,0,5,"[0.2, 1.0, 2.0, 5.0]",[0],[1],0.0,1000,"[""gd"", ""L-BFGS-B"", ""logistic""]",1
8,258d2d39-4a07-4ae6-8299-2e445d07b336,Default Experiment,44.692566,7,2023-03-25 12:30:28,0,5,"[0.2, 1.0, 2.0, 5.0]",[0],[1],2.0,1000,"[""gd"", ""L-BFGS-B"", ""logistic""]",1


In [48]:
# extract and print the top experiment_id
experiment_id = df_experiments.iloc[0]["experiment_id"]
print(experiment_id)

e771baf8-dd50-467e-9262-2346ca9f0a0d


In [49]:
df_state_evolution[df_state_evolution["experiment_id" ] == experiment_id]

,id,code_version,duration,experiment_id,generalization_error,date,sigma,q,m,initial_condition,rho_w_star,alpha,epsilon,tau,lam,abs_tol,min_iter,max_iter,blend_fpe,int_lims


In [50]:
erm = df_erm[df_erm["experiment_id" ] == experiment_id]
erm[erm["chosen_minimizer"] == "logistic"]

,id,duration,code_version,experiment_id,Q,rho,m,cosb,epsilon,lam,generalization_error_erm,generalization_error_overlap,date,chosen_minimizer,training_error,d,tau,alpha,test_loss
356,eff9af70-def7-4ba6-b34e-d6e779620510,4.298904,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.033987,1.081111,0.059383,0.309794,0.0,1.0,0.40014,0.399740,2023-03-25 12:43:20,logistic,0.0100,1000,0.0,0.2,0.674655
363,9ce7be2a-260d-46ed-9de3-ec11501d2f7e,4.580292,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.033056,0.987342,0.056756,0.314162,0.0,1.0,0.39860,0.398277,2023-03-25 12:43:22,logistic,0.0100,1000,0.0,0.2,0.674230
365,ec02afd2-6f9e-450e-984f-3a823a1ee861,2.873655,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.031985,1.100675,0.064629,0.344445,0.0,1.0,0.38946,0.388067,2023-03-25 12:43:26,logistic,0.0000,1000,0.0,0.2,0.672642
366,6c4f0f71-17c5-4779-8548-9c026016d374,2.910263,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.032281,1.031078,0.058271,0.319395,0.0,1.0,0.39527,0.396520,2023-03-25 12:43:26,logistic,0.0050,1000,0.0,0.2,0.673895
367,f82e2bdd-a71c-4254-b6e2-289fae22682a,2.797946,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.030034,1.007679,0.060015,0.344976,0.0,1.0,0.39074,0.387887,2023-03-25 12:43:26,logistic,0.0150,1000,0.0,0.2,0.673678
373,211d4713-ac95-4d66-8b95-6b1912627dc1,2.893064,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.177693,0.960381,0.273984,0.663238,0.0,1.0,0.26628,0.269182,2023-03-25 12:43:31,logistic,0.0360,1000,0.0,1.0,0.602294
375,ef553ed0-c15b-4ca1-a20d-4f31acc1c933,3.007569,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.181332,0.983968,0.276354,0.654242,0.0,1.0,0.27190,0.272988,2023-03-25 12:43:31,logistic,0.0380,1000,0.0,1.0,0.603815
377,f76e464e-d8b2-44c5-9be1-e01a7dad1c40,2.988414,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.176066,0.906851,0.257970,0.645600,0.0,1.0,0.27858,0.276607,2023-03-25 12:43:31,logistic,0.0320,1000,0.0,1.0,0.607125
380,754fb4b8-c121-4294-b62b-56a34f6782d1,2.808681,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.173491,1.028704,0.278711,0.659735,0.0,1.0,0.27193,0.270669,2023-03-25 12:43:36,logistic,0.0400,1000,0.0,1.0,0.604734
382,e7f4d24e-a4af-4a78-8b15-d5f3183aa595,2.832602,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.177649,0.972647,0.271906,0.654125,0.0,1.0,0.27296,0.273037,2023-03-25 12:43:36,logistic,0.0400,1000,0.0,1.0,0.605344


In [51]:
erm[erm["chosen_minimizer"] == "gd"]

,id,duration,code_version,experiment_id,Q,rho,m,cosb,epsilon,lam,generalization_error_erm,generalization_error_overlap,date,chosen_minimizer,training_error,d,tau,alpha,test_loss
359,1d4a32df-a1a9-47a9-a410-3d2e879f5582,6.917801,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.036778,0.963139,0.063649,0.338181,0.0,1.0,0.39068,0.390188,2023-03-25 12:43:22,gd,0.0000,1000,0.0,0.2,0.671848
360,17ff6706-7676-46a5-869f-9e8de58b6b2a,4.320332,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.036023,0.970383,0.060917,0.325820,0.0,1.0,0.39669,0.394360,2023-03-25 12:43:22,gd,0.0000,1000,0.0,0.2,0.673303
362,cd1778f1-c037-4bae-a450-91ed262ac922,4.435365,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.030359,0.992618,0.056590,0.325993,0.0,1.0,0.39708,0.394301,2023-03-25 12:43:22,gd,0.0000,1000,0.0,0.2,0.674599
368,51ed17b6-8e74-4342-9776-c5d1a1f8eea0,2.922942,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.036847,1.023982,0.063185,0.325288,0.0,1.0,0.39493,0.394539,2023-03-25 12:43:27,gd,0.0000,1000,0.0,0.2,0.672781
369,fcdba382-2369-4287-af0a-7b36f28ff6cd,2.877890,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.030578,0.918991,0.055277,0.329750,0.0,1.0,0.39441,0.393036,2023-03-25 12:43:27,gd,0.0000,1000,0.0,0.2,0.674328
371,70b6183a-496e-4f6e-9b8f-4e46ea591920,3.431311,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.179214,0.939138,0.274780,0.669783,0.0,1.0,0.26600,0.266387,2023-03-25 12:43:27,gd,0.0330,1000,0.0,1.0,0.601757
378,09d852cc-abf8-407e-9a4d-ea54ff0e52ec,3.573791,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.181265,0.957974,0.282119,0.677016,0.0,1.0,0.26457,0.263273,2023-03-25 12:43:32,gd,0.0350,1000,0.0,1.0,0.600806
379,fbab8f96-353d-4995-b072-a1ab5711ab53,3.617582,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.174720,0.990566,0.286252,0.688074,0.0,1.0,0.25943,0.258456,2023-03-25 12:43:32,gd,0.0330,1000,0.0,1.0,0.600234
381,5b157ce6-e41d-4f32-808e-c49fc59e95fc,3.162441,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.178030,0.967471,0.271590,0.654407,0.0,1.0,0.27256,0.272918,2023-03-25 12:43:36,gd,0.0380,1000,0.0,1.0,0.605481
386,2c214b03-1cf6-4381-a719-b335b4d86d05,3.257284,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.384843,1.021811,0.511997,0.816470,0.0,1.0,0.19609,0.195928,2023-03-25 12:43:36,gd,0.0350,1000,0.0,2.0,0.538040


In [52]:
erm[erm["chosen_minimizer"] == "L-BFGS-B"]

,id,duration,code_version,experiment_id,Q,rho,m,cosb,epsilon,lam,generalization_error_erm,generalization_error_overlap,date,chosen_minimizer,training_error,d,tau,alpha,test_loss
357,1474ae37-0e4e-42e1-819a-41a8fb93f593,4.456638,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.032707,0.983764,0.059604,0.332281,0.0,1.0,0.39217,0.392182,2023-03-25 12:43:21,L-BFGS-B,0.0000,1000,0.0,0.2,0.673257
358,09bb47e3-2f08-4165-b5b5-94c1e960538f,4.298436,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.031599,1.012520,0.058697,0.328157,0.0,1.0,0.39077,0.393572,2023-03-25 12:43:21,L-BFGS-B,0.0000,1000,0.0,0.2,0.673326
361,d4a1dbab-183a-433e-9a67-586e1a54daef,4.299042,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.031393,0.961438,0.062612,0.360392,0.0,1.0,0.38185,0.382643,2023-03-25 12:43:22,L-BFGS-B,0.0000,1000,0.0,0.2,0.671693
364,a4be7489-ae33-4a3f-a324-79352d44d229,2.776170,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.032398,0.978171,0.059626,0.334943,0.0,1.0,0.39144,0.391283,2023-03-25 12:43:26,L-BFGS-B,0.0000,1000,0.0,0.2,0.672773
370,5112b39b-e90f-4fdc-9d4f-2c049790aeb8,3.250064,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.033834,1.007658,0.067642,0.366337,0.0,1.0,0.37878,0.380612,2023-03-25 12:43:27,L-BFGS-B,0.0000,1000,0.0,0.2,0.670076
372,5a0be9d7-78d2-4335-b9db-6664674d069a,2.927937,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.179398,1.066074,0.280501,0.641406,0.0,1.0,0.27734,0.278351,2023-03-25 12:43:31,L-BFGS-B,0.0400,1000,0.0,1.0,0.605740
374,3d09ea61-e3fd-4274-be91-4e278bfc9c71,2.987275,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.178651,1.048371,0.285812,0.660420,0.0,1.0,0.26878,0.270378,2023-03-25 12:43:31,L-BFGS-B,0.0300,1000,0.0,1.0,0.603767
376,660431b9-704c-45b9-a8cf-14caa9caef39,3.097832,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.174103,1.040260,0.282984,0.664949,0.0,1.0,0.26457,0.268453,2023-03-25 12:43:31,L-BFGS-B,0.0420,1000,0.0,1.0,0.602674
383,6fbf7467-5e55-4486-bd98-2c886563d4e0,2.969160,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.178747,0.925289,0.264650,0.650749,0.0,1.0,0.27349,0.274455,2023-03-25 12:43:36,L-BFGS-B,0.0300,1000,0.0,1.0,0.605681
384,3542fb10-c4a5-435f-a60b-c095ad92a8d1,3.046048,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.182953,1.014368,0.286957,0.666115,0.0,1.0,0.26738,0.267956,2023-03-25 12:43:36,L-BFGS-B,0.0320,1000,0.0,1.0,0.601711


In [48]:
# merge df_erm and df_state_evolution on experiment id efd6ee50-96f6-4ddd-9a84-ce6c74469637
erm = df_erm[df_erm["experiment_id" ] == "efd6ee50-96f6-4ddd-9a84-ce6c74469637"]
state_evolution = df_state_evolution[df_state_evolution["experiment_id" ] == "efd6ee50-96f6-4ddd-9a84-ce6c74469637"]
# print the number of rows in each dataframe
print("erm: %d, state_evolution: %d" % (erm.shape[0],state_evolution.shape[0]))
# round alpha and epsilon to 2 decimals
erm["alpha"] = erm["alpha"].round(2)
erm["epsilon"] = erm["epsilon"].round(2)
state_evolution["alpha"] = state_evolution["alpha"].round(2)
state_evolution["epsilon"] = state_evolution["epsilon"].round(2)
# keep alpha, epsilon and the generalization errors
erm = erm[["alpha","epsilon","generalization_error_erm","generalization_error_overlap"]]
state_evolution = state_evolution[["alpha","epsilon","generalization_error"]]

# join the dataframes on alpha an epsilon and add suffixes to the columns
df = erm.merge(state_evolution, on=["alpha","epsilon"], suffixes=("_erm","_state_evolution"))
# print the number of rows in the joined dataframe
print("joined: %d" % df.shape[0])
df

erm: 6, state_evolution: 6
joined: 6


/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_6324/4173925508.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_evolution["alpha"] = state_evolution["alpha"].round(2)
/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_6324/4173925508.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_evolution["epsilon"] = state_evolution["epsilon"].round(2)


,alpha,epsilon,generalization_error_erm,generalization_error_overlap,generalization_error
0,0.64,0.02,0.46955,0.440456,0.122996
1,0.64,0.00,0.46935,0.416841,0.123619
2,2.28,0.02,0.44400,0.351962,0.066441
3,5.00,0.00,0.41380,0.300245,0.045324
4,5.00,0.02,0.41235,0.287456,0.045062
5,2.28,0.00,0.44595,0.386968,0.066818
